In [ ]:
import cv2, os
import numpy as np
from keras.models import load_model
import pymysql
import pandas as pd
import time
import warnings
import datetime
from Invester import graph, func
warnings.filterwarnings('ignore')

class model:
    def __init__(self):
        """DB 연결"""
        self.conn = pymysql.connect(host='localhost', port = 3306, user='root', password='1234', db='INVESTAR', charset='utf8', 
                          autocommit = True, cursorclass = pymysql.cursors.DictCursor)

        with self.conn.cursor() as curs:
            sql = """ 
            CREATE TABLE IF NOT EXISTS result ( 
                last_update TIME,
                code VARCHAR(20),
                nochange FLOAT,
                up FLOAT,
                down FLOAT,
                PRIMARY KEY (code))
                """
            curs.execute(sql)

        self.conn.commit()
        self.codes = {}
        """company_info 테이블에서 읽어와서 companyData와 codes에 저장"""
        sql = "SELECT * FROM company_info"
        companyInfo = pd.read_sql(sql, self.conn)
        for idx in range(len(companyInfo)):
            self.codes[companyInfo['code'].values[idx]] = companyInfo['company'].values[idx]
        
        # 실시간 이미지 저장될 폴더
        if not os.path.exists('./static/daily'):
            os.mkdir('./static/daily')
    
    # 이미지 파일 리사이즈
    def Dataization(self, img_path):
            image_w = 64
            image_h = 64
            img = cv2.imread(img_path)
            img = cv2.resize(img, None, fx=image_w/img.shape[1], fy=image_h/img.shape[0])
            return (img/256)
    
    # 예측
    def pred(self, code):        
        image_dir = "./static/daily/"
        file = image_dir + f'{code}.png'
        test = []
        test.append(self.Dataization(file))
        test = np.array(test)
        model = load_model('img_cnn.h5')
        predict = model.predict(test)
        current_time = datetime.datetime.now().time()
        with self.conn.cursor() as curs:
            sql = f"REPLACE INTO result (last_update, code, nochange, up, down) VALUES ('{current_time}', '{code}','{round(predict[0][0]*100,2)}','{round(predict[0][1]*100,2)}','{round(predict[0][0]*100,2)}')"
            curs.execute(sql)
            self.conn.commit()
        print(current_time, code, predict)
    
    # 반전형인지 확인
    def main(self, code):
        with self.conn.cursor() as curs:
            # 최근 20분 동안의 데이터
            sql = f"SELECT Date, Open, High, Low, Close from daily_stocks WHERE code = {code} AND Date >= date_sub(now(), interval 20 minute)"
            curs.execute(sql)
            rows = curs.fetchall()
        if len(rows) >= 2:
            full_rows_df = pd.DataFrame(rows)
            rows_df = pd.DataFrame(rows[:-1])
            row = rows[-1]
            lst = {'D':'직선형'}
            
            lst = func.HL(rows_df, lst)
            lst['D'] = func.direction(lst)
            lst = func.new_HL(lst, row, len(full_rows_df))
            new_D = func.direction(lst)

            if lst['D'] != new_D:
                graph.train_candle(full_rows_df,code,'static')
                self.pred(code)
        
    def all_code(self):
        for code in self.codes:
            self.main(code)

if __name__ == '__main__':
    mode = model()
    while True:
        mode.all_code()
        # 1분마다 재실행
        time.sleep(60)

003530
[[0.4234023  0.3784     0.19819774]]
003920
[[0.53296936 0.21523644 0.25179422]]
005440
[[0.6072212  0.11042678 0.2823521 ]]
005490
[[0.6984202  0.09780523 0.20377451]]
007610
[[0.4254142  0.40522215 0.16936369]]
008490
[[0.6186016  0.09318774 0.2882106 ]]
012280
[[0.5328253  0.21532659 0.2518481 ]]
012690
[[0.48231313 0.28258118 0.2351057 ]]
014820
[[0.731506   0.0942588  0.17423517]]
026960
[[0.711781   0.09814036 0.19007862]]
101060
[[0.53296936 0.21523644 0.25179422]]
129260
[[0.53296936 0.21523644 0.25179422]]
